In [1]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session

In [2]:
bucket_name = 'projectonebucket'
my_region = boto3.session.Session().region_name
print(my_region)

us-east-2


In [3]:
prefix = 'xgboost-as-a-built-in-algo'
output_path = 's3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://projectonebucket/xgboost-as-a-built-in-algo/output


In [10]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('success: downloaded dataset file')
except Exception as e:
    print('Data load error: ',e)
try:
    model_data = pd.read_csv('./bank_clean.csv', index_col=0)
    print('success: data loaded into dataframe')
except Exception as e:
    print('Data load error: ', e)

success: downloaded dataset file
success: data loaded into dataframe


In [11]:
import numpy as np
train_data, test_data = np.split( model_data.sample(frac=1, random_state=1729), [int(0.7*len(model_data))] )
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [32]:
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)],axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix,'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [33]:
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)],axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix,'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

In [34]:
#container = get_image_uri(boto3.Session().region_name, 'xgboost', repo_version='1.2-2')
container = sagemaker.image_uris.retrieve('xgboost', boto3.Session().region_name, version='1.2-2')

In [35]:
hyperparameters = { 'max_depth':'5', 'eta':'0.2', 'gamma':'4', 'min_child_weight':'6', 'subsample':'0.7', 
                   'objective':'binary:logistic', 'num_round':50 }

In [36]:
estimator = sagemaker.estimator.Estimator(image_uri=container, hyperparameters=hyperparameters,
                                         role=sagemaker.get_execution_role(), instance_count=1, instance_type='ml.m5.2xlarge',
                                         volume_size=5, output_path=output_path, use_spot_instances=True, max_run=300,
                                         max_wait=600)

In [37]:
estimator.fit({'train':s3_input_train, 'validation':s3_input_test})

2021-06-12 18:55:02 Starting - Starting the training job...
2021-06-12 18:55:04 Starting - Launching requested ML instancesProfilerReport-1623524102: InProgress
......
2021-06-12 18:56:28 Starting - Preparing the instances for training.........
2021-06-12 18:57:57 Downloading - Downloading input data
2021-06-12 18:57:57 Training - Downloading the training image..[2021-06-12 18:58:13.260 ip-10-0-126-83.us-east-2.compute.internal:1 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-06-12:18:58:13:INFO] Imported framework sagemaker_xgboost_container.training
[2021-06-12:18:58:13:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2021-06-12:18:58:13:INFO] No GPUs detected (normal if no gpus installed)
[2021-06-12:18:58:13:INFO] Running XGBoost Sagemaker in algorithm mode
[2021-06-12:18:58:13:INFO] Determined delimiter of CSV input is ','
[2021-06-12:18:58:13:INFO] Determined delimiter of CSV input is ','
[2021-06-12:18:58:13:I

In [38]:
xgb_predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

---------------!

In [47]:
#from sagemaker.predictor import csv_serializer
from sagemaker.serializers import CSVSerializer
#sagemaker.serializers.CSVSerializer()

test_data_array = test_data.drop( ['y_no','y_yes'], axis=1 ).values
#xgb_predictor.content_type = 'text/csv'
#xgb_predictor.serializer = csv_serializer
xgb_predictor.serializer = CSVSerializer('text/csv')

predictions = xgb_predictor.predict(test_data_array).decode('utf-8')
predictions_array = np.fromstring(predictions[1:], sep=',')
print(predictions_array.shape)

(12357,)


In [48]:
predictions_array

array([0.05214286, 0.05660191, 0.05096195, ..., 0.03436061, 0.02942475,
       0.03715819])

In [49]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10785)    34% (151)
Purchase        9% (1124)     66% (297) 



In [54]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint_name)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': 'ATPM2228XAGKBVDR',
   'HostId': 'NtVbgdVLiuoH9TtmcFGX40E9tIOXEaY/wE/ASK6lE4n2cgDfOwCDnxywO1Cw9mJQzFcL1Rogou4=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'NtVbgdVLiuoH9TtmcFGX40E9tIOXEaY/wE/ASK6lE4n2cgDfOwCDnxywO1Cw9mJQzFcL1Rogou4=',
    'x-amz-request-id': 'ATPM2228XAGKBVDR',
    'date': 'Sat, 12 Jun 2021 19:33:14 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2021-06-12-18-55-02-039/profiler-output/framework/training_job_end.ts'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2021-06-12-18-55-02-039/rule-output/ProfilerReport-1623524102/profiler-output/profiler-reports/StepOutlier.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2021-06-12-18-55-02-039/rule-output/ProfilerReport-1623524102/profiler-output/p